# Test Environment
data\
    talent.xlsx\
    waffen.xlsx\
    zauber.xlsx

einheit\
    held1.xlsx\
    held1_inventar.xlsx

In [116]:
# Modul um Würfel zu simulieren
import random
random.seed(1321345589)

# Modul zum vereinfachten Auslesen und Bearbeiten von Datentabellen
import pandas as pd

# Dies und Das
import regex as re

In [117]:
def w20():
    """
    Simple Simulation eines W20 Würfels.
    Benötigt keinen Input und gibt immer einen Wert zwischen 1 und 20 zurück.
    """

    return random.randint(1, 20)


def w6(anzahl: int = 1):
    """
    Simple Simulation eines W6 Würfels.
    Benötigt keinen Input und gibt immer einen Wert zwischen 1 und 6 zurück.
    """

    assert anzahl > 0

    wuerfe = [random.randint(1, 6) for _ in range(anzahl)]

    return sum(wuerfe)


def datei_zu_dataframe(path, index_column=0):
    """
    
    """

    dataframe = pd.read_excel(path, index_col=index_column)
    dataframe.dropna(inplace=True)

    if "Eigenschaftsprobe" in dataframe.columns:
        dataframe["Eigenschaftsprobe"] = dataframe.Eigenschaftsprobe.apply(lambda x: x.split("/"))

    return dataframe


def datei_zu_dict(path):
    """
    Liest excel Tabelle mit Informationen zum Helden ein, damit diese als
    Klassen-Attribute übergeben werden können.
    
    : name : Dateiname bzw. Name des Helden, Pfad in /einheit/name
    : return : dict
    """

    frame = datei_zu_dataframe(path)
    framedict = frame.T.to_dict(orient="records")[0]

    return framedict


def zeile_zu_dict(frame, zeile: str):
    """
    
    """

    framedict = frame.T[zeile].to_dict()
    framedict["Waffe"] = zeile

    return framedict

In [118]:
# Einlesen der Datentabelle zu den Fertigkeiten
dsa5_talente = datei_zu_dataframe("data/dsa5_talente.xlsx", index_column=1)

dsa5_waffen = datei_zu_dataframe("data/waffen.xlsx")
dsa5_ruestungen = datei_zu_dataframe("data/ruestungen.xlsx")
dsa5_zauber = datei_zu_dataframe("data/zauber.xlsx")

# Glossar einlesen und mit dem gespiegelten selbst erweitern
glossar = datei_zu_dict("data/glossar.xlsx")
glossar = glossar | {val: key for key, val in glossar.items()}

# Einlesen von Helden
margot_data = datei_zu_dict("data/held_margot.xlsx")
margot_inv = datei_zu_dataframe("data/held_margot_inventar.xlsx")
elsbeth_data = datei_zu_dict("data/held_elsbeth.xlsx")
elsbeth_inv = datei_zu_dataframe("data/held_elsbeth_inventar.xlsx")

In [119]:
elsbeth_inv[elsbeth_inv["Art"] == "Waffe"].iloc[0].name

'Breitschwert'

In [120]:
class Einheit():
    def __init__(self, heldenname):
        if f"data/held_{heldenname}.xlsx" is not None:
            file = datei_zu_dict(f"data/held_{heldenname}.xlsx")

            for key, value in file.items():
                setattr(self, key, value)
        
        self.inventar = datei_zu_dataframe(f"data/held_{heldenname}_inventar.xlsx")
        
        self.waffe_ausruesten()
        self.ruestung = None
        self.zauber = None
    

    def waffe_ausruesten(self, waffe=None):
        """
        
        """

        if waffe == None:
            waffe = self.inventar[self.inventar["Art"] == "Waffe"].iloc[0].name

        assert waffe in dsa5_waffen.T.columns.to_list()

        framedict = zeile_zu_dict(dsa5_waffen, waffe)
        self.haupthand = framedict


    def ruestung_tragen(self, ruestung=None):
        """
        
        """

        if ruestung == None:
            ruestung = self.inventar[self.inventar["Art"] == "Rüstung"].iloc[0].name

        assert ruestung in dsa5_ruestungen.T.columns.to_list()

        framedict = zeile_zu_dict(dsa5_ruestungen, ruestung)
        self.ruestung = framedict
     
    
    def attacke(self, other=None):
        """
        
        """

        versuch = w20()
        probe = getattr(self, self.haupthand["L+S"])
        erfolg = versuch <= probe
        kritischer_erfolg = versuch == 1

        if kritischer_erfolg:
            schaden = w6(self.haupthand["TP"])
        
        else:
            pass


    def parade(self, other):
        pass


    def eigenschaftsprobe(self, probe: str, bias: int=0):
        wurf = w20()

        if wurf > getattr(self, glossar[probe]) + bias:
            print(f"Eigenschaftsprobe misslungen! Der nötige {probe}-Wert von {getattr(self, glossar[probe])} wurde durch {wurf} nicht unterworfen!")
        
        else:
            print(f"Eigenschaftsprobe geschafft! Der nötige {probe}-Wert von {getattr(self, glossar[probe])} wurde durch {wurf} unterworfen!")
    

    def fertigkeitsprobe(self, probe: str, bias: int=0):
        """
        
        """

        ausgleich = random.randint(3, 7)
        print(f'Starte Fertigkeitsprobe {probe} auf {dsa5_talente["Eigenschaftsprobe"][probe]}, Ausgleich: {ausgleich}')

        for i, eigenschaft in enumerate(dsa5_talente["Eigenschaftsprobe"][probe]):
            wurf = w20()

            ziel = getattr(self, eigenschaft) + bias
            wurfdifferenz = ziel - wurf

            if wurfdifferenz < 0:
                ausgleich += wurfdifferenz
            
                if ausgleich < 0:
                    print(f'\nFertigkeitsprobe misslungen!')

                    return
        
        print(f'\nFertigkeitsprobe geglückt! Übriger Ausgleich bei {ausgleich}.')

In [121]:
margot = Einheit("margot")
elsbeth = Einheit("elsbeth")

In [122]:
margot.ruestung